In [1]:
import utils as ut

In [2]:
%matplotlib inline

In [3]:
from sklearn.feature_extraction.text import TfidfTransformer

In [4]:
from sklearn.feature_extraction.text import CountVectorizer#, TfidfVectorizer

In [5]:
file_to_read = ut.os.path.join(ut.DATA_DIR, 'cleaned_train.tsv')
df_cleaned = ut.pd.read_table(file_to_read, encoding='utf-8', \
                                    sep="\t")

In [6]:
df_cleaned['tag'] = \
    df_cleaned['tag'].apply(ut.change_string_to_int_list)

df_cleaned.shape

(10578, 4)

In [51]:
df_cleaned.iloc[0, 1]

'interfacesportsnetwork rj45 yesgener informationproduct type pdumanufactur part number pdu20mhvt10atmanufactur websit address httpwwwcyberpowersystemscommanufactur cyberpow system incproduct model pdu20mhvt10atproduct name pdu20mhvt10at meter at pdu 200240v 20a 1u 10outlet 2 l620pproduct seri meteredbrand name cyberpowermanagementmanag http tcpip udp dhcp snmp snmpv3 ipv4 ipv6 ntp smtp miscellaneouspackag content pdu20mhvt10at meter at pdu 200240v 20a 1u 10outlet 2 l620 mount bracket 1 set cord retent tray cord organ user manual compat data center server network devic telecom equip cyberpow upss pr5000lcdrtxl5u pr6000lcdrtxl5u ol6000rt3u ol6000rt3utf ol8000rt3utf ol10000rt3utf certif standard ul609501 certifi etl usa csa c222canada fcc class green compliant yesgreen complianc certificateauthor rohsphys characteristicsweight approxim 1146 lbform factor horizont rackmountrack height 1ucord length 10 ftheight 18width 171depth 93number cordcabl 2power descriptioninput voltag 230 v acoutpu

In [8]:
df_cleaned.loc[df_cleaned['Product Long Description'].isnull()]

,Item Class ID,Product Long Description,Product Name,tag


In [9]:
df_cleaned['Product Long Description'].loc[(df_cleaned['Product Long Description'].isnull())]

Series([], Name: Product Long Description, dtype: object)

In [10]:
vectorizer = CountVectorizer(analyzer = "word",
                                 tokenizer = None,
                                 preprocessor = None,
                                 ngram_range = (1, 1),
                                 strip_accents = 'unicode',
                                 max_features = 1000)

In [11]:
%time feature_matrix_bow = vectorizer.fit_transform(df_cleaned['Product Long Description'])
feature_matrix_bow

CPU times: user 1.07 s, sys: 16 ms, total: 1.09 s
Wall time: 1.09 s


<10578x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 346701 stored elements in Compressed Sparse Row format>

In [40]:
tfidf_transformer = TfidfTransformer()
%time feature_matrix_tfidf = tfidf_transformer.fit_transform( \
                                            feature_matrix_bow)
feature_matrix_tfidf.shape

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 7.8 ms


(10578, 1000)

In [21]:
X_bog = ut.np.concatenate((feature_matrix_bow.toarray(), dummy_item_class_id), axis=1)

In [41]:
X_tfidf = ut.np.concatenate( \
            (feature_matrix_tfidf.toarray(), dummy_item_class_id), axis=1)

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.datasets import make_multilabel_classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
# from sklearn.preprocessing import LabelBinarizer
# from sklearn.decomposition import PCA
# from sklearn.cross_decomposition import CCA

In [14]:
mlb = MultiLabelBinarizer()
dummy_item_class_id = mlb.fit_transform(df_cleaned['Item Class ID'])
dummy_tags = mlb.fit_transform(df_cleaned['tag'])

In [15]:
dummy_item_class_id.shape

(10578, 13)

# Classification - SVM

In [16]:
def my_multi_svc(X_train, Y_train):
    
    classif = OneVsRestClassifier(SVC(kernel='linear'))
    %time classif.fit(X_train, Y_train)
    
    return classif    

## DUMMY ONLY

In [17]:
classif_dummy = my_multi_svc(dummy_item_class_id, dummy_tags)

CPU times: user 5.07 s, sys: 308 ms, total: 5.38 s
Wall time: 5.41 s


In [18]:
classif_dummy.score(dummy_item_class_id, dummy_tags)

0.027509926262053317

## BAG OF WORDS ONLY

In [19]:
classif_bow = my_multi_svc(feature_matrix_bow.toarray(), dummy_tags)

CPU times: user 4min 26s, sys: 1.56 s, total: 4min 28s
Wall time: 4min 28s


In [24]:
%time classif_bow.score(feature_matrix_bow.toarray(), dummy_tags)

CPU times: user 2min 35s, sys: 672 ms, total: 2min 36s
Wall time: 2min 36s


0.83059179429003593

## BAG OF WORDS + DUMMY

In [47]:
classif_bow_dummy = my_multi_svc(X_bog, dummy_tags)

CPU times: user 4min 11s, sys: 1.09 s, total: 4min 12s
Wall time: 4min 12s


In [48]:
%time classif_bow_dummy.score(X_bog, dummy_tags)

CPU times: user 2min 30s, sys: 612 ms, total: 2min 31s
Wall time: 2min 31s


0.84968803176403862

## TFIDF VECTORS ONLY

In [49]:
classif_tfidf = my_multi_svc(feature_matrix_tfidf.toarray(), dummy_tags)

CPU times: user 3min 12s, sys: 416 ms, total: 3min 13s
Wall time: 3min 13s


In [50]:
%time classif_tfidf.score(feature_matrix_tfidf.toarray(), dummy_tags)

CPU times: user 2min 45s, sys: 156 ms, total: 2min 46s
Wall time: 2min 46s


0.68595197579882772

## TFIDF + DUMMY

In [ ]:
classif_tfidf_dummy = my_multi_svc(X_tfidf, dummy_tags)

In [ ]:
%time classif_tfidf_dummy.score( X_tfidf, dummy_tags)

# Classification - Bayes

In [30]:
from sklearn.naive_bayes import MultinomialNB
# from sklearn.naive_bayes import GaussianNB
# from sklearn.naive_bayes import BernoulliNB

def my_NB(X_train, Y_train):
    
    classif = OneVsRestClassifier( MultinomialNB())
    %time classif.fit(X_train, Y_train)
    
#     %time clf = MultinomialNB().fit(X_train, Y_train) 
    
    return classif

## DUMMY ONLY

In [31]:
classif_nb_dummy = my_NB(dummy_item_class_id, dummy_tags)

CPU times: user 480 ms, sys: 4 ms, total: 484 ms
Wall time: 146 ms


In [32]:
%time classif_nb_dummy.score(dummy_item_class_id, dummy_tags)

0.024768387218755909

## BAG OF WORDS ONLY

In [33]:
classif_nb_bow = my_NB(feature_matrix_bow.toarray(), dummy_tags)

CPU times: user 5.08 s, sys: 740 ms, total: 5.82 s
Wall time: 1.5 s


In [34]:
%time classif_nb_bow.score(feature_matrix_bow.toarray(), dummy_tags)

CPU times: user 4.21 s, sys: 644 ms, total: 4.86 s
Wall time: 1.21 s


0.34354320287388923

## BAG OF WORDS + DUMMY

In [35]:
classif_nb_bog_dummy = my_NB(X_bog, dummy_tags)

CPU times: user 5.43 s, sys: 672 ms, total: 6.1 s
Wall time: 1.58 s


In [36]:
%time classif_nb_bog_dummy.score(X_bog, dummy_tags)

CPU times: user 4.3 s, sys: 672 ms, total: 4.98 s
Wall time: 1.25 s


0.34855360181508793

## TFIDF VECTORS ONLY

In [37]:
classif_nb_tfidf = my_NB(feature_matrix_tfidf.toarray(), dummy_tags)

CPU times: user 2.56 s, sys: 40 ms, total: 2.6 s
Wall time: 675 ms


In [38]:
%time classif_nb_tfidf.score(feature_matrix_tfidf.toarray(), dummy_tags)

CPU times: user 1.81 s, sys: 60 ms, total: 1.87 s
Wall time: 468 ms


0.5128568727547741

## TFIDF + DUMMY

In [42]:
classif_nb_tfidf_dummy = my_NB(X_tfidf, dummy_tags)

CPU times: user 2.74 s, sys: 64 ms, total: 2.8 s
Wall time: 726 ms


In [43]:
%time classif_nb_tfidf_dummy.score( X_tfidf, dummy_tags)

CPU times: user 1.69 s, sys: 8 ms, total: 1.7 s
Wall time: 424 ms


0.51540934013991302

# Classification TREE

## DUMMY ONLY

In [25]:
from sklearn.tree import DecisionTreeClassifier

def my_Tree(X_train, Y_train):
    
    classif = OneVsRestClassifier(DecisionTreeClassifier(random_state=0))
    %time classif.fit(X_train, Y_train)
    
#     %time clf = MultinomialNB().fit(X_train, Y_train) 
    
    return classif

In [26]:
classif_tree_dummy = my_Tree(dummy_item_class_id, dummy_tags)

CPU times: user 108 ms, sys: 0 ns, total: 108 ms
Wall time: 109 ms


In [27]:
%time classif_tree_dummy.score(dummy_item_class_id, dummy_tags)

CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 36.1 ms


0.040839478162223483

## BAG OF WORDS ONLY

In [28]:
classif_tree_bow = my_Tree(feature_matrix_bow.toarray(), dummy_tags)

CPU times: user 1min 27s, sys: 272 ms, total: 1min 27s
Wall time: 1min 27s


In [29]:
%time classif_tree_bow.score(feature_matrix_bow.toarray(), dummy_tags)

0.98355076574021549

## BAG OF WORDS + DUMMY

In [ ]:
classif_tree_bog_dummy = my_Tree(X_bog, dummy_tags)

In [ ]:
%time classif_tree_bog_dummy.score(X_bog, dummy_tags)

## TFIDF VECTORS ONLY

In [44]:
classif_tree_tfidf = my_Tree(feature_matrix_tfidf.toarray(), dummy_tags)

CPU times: user 2min 15s, sys: 304 ms, total: 2min 15s
Wall time: 2min 15s


In [45]:
%time classif_tree_tfidf.score(feature_matrix_tfidf.toarray(), dummy_tags)

CPU times: user 496 ms, sys: 336 ms, total: 832 ms
Wall time: 827 ms


0.98317262242389869

## TFIDF + DUMMY

In [ ]:
classif_tree_tfidf_dummy = my_Tree(X_tfidf, dummy_tags)

In [ ]:
%time classif_tree_tfidf_dummy.score( X_tfidf, dummy_tags)